# Foundamental Course — Week 4 Practice (Starter Notebook)

---

## Pre-study (Self-learn)

Foundamental Course assumes Self-learn is complete. If you need a refresher on production constraints and operational habits:

- [Foundamental Course Pre-study index](../PRESTUDY.md)
- [Self-learn — Chapter 5: Resource Monitoring and Containerization](../../self_learn/Chapters/5/Chapter5.md)

---

Starter LLM client skeleton: timeouts, retries, caching, and logs.

## What success looks like (end of practice)

- You can make a call through `llm_call(...)` and see retry behavior.
- You can demonstrate a cache hit.
- You can point to at least one raw failure saved under `output/`.

### Checkpoint

- `llm_call('hello', ...)` returns a response dict.
- Re-running the same call yields a cache hit.

## References (docs)
- `requests` timeouts: https://requests.readthedocs.io/en/latest/user/quickstart/#timeouts
- Tenacity: https://tenacity.readthedocs.io/
- Python `logging`: https://docs.python.org/3/library/logging.html
- HTTP 429 Too Many Requests (MDN): https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429
- Twelve-Factor App: https://12factor.net/

## Setup

This notebook demonstrates patterns without requiring a real API key.
Replace `fake_provider_call(...)` with a real provider call later.


In [ ]:
import hashlib
import json
import logging
import time
from dataclasses import dataclass
from typing import Any, Dict

import requests
from tenacity import retry, stop_after_attempt, wait_exponential


In [ ]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s %(message)s')
logger = logging.getLogger('llm_client')

CACHE: Dict[str, Any] = {}


## Stable cache keys

Cache keys should include everything that changes output: model, prompt, temperature, etc.


In [ ]:
def make_cache_key(payload: dict) -> str:
    raw = json.dumps(payload, sort_keys=True, ensure_ascii=False).encode('utf-8')
    return hashlib.sha256(raw).hexdigest()


## Provider call stub

Simulate transient failures so you can test retries/timeouts.


In [ ]:
def fake_provider_call(payload: dict, timeout_s: float) -> dict:
    if payload.get('force_error'):
        raise requests.Timeout('Simulated timeout')
    time.sleep(0.05)
    return {
        'text': 'echo: ' + str(payload.get('prompt', '')) ,
        'model': payload.get('model', 'fake'),
    }


## LLM client skeleton

Implements timeout + retry/backoff + caching + logs.


In [ ]:
@dataclass
class LLMConfig:
    model: str = 'fake-model'
    timeout_s: float = 10.0
    max_retries: int = 3

cfg = LLMConfig()
cfg


In [ ]:
def llm_call(prompt: str, *, config: LLMConfig, force_error: bool = False) -> dict:
    payload = {
        'model': config.model,
        'prompt': prompt,
        'force_error': force_error,
    }
    cache_key = make_cache_key(payload)
    if cache_key in CACHE:
        logger.info('cache_hit')
        return CACHE[cache_key]

    @retry(stop=stop_after_attempt(config.max_retries), wait=wait_exponential(multiplier=0.5, min=0.5, max=4.0))
    def _call_once():
        t0 = time.time()
        try:
            return fake_provider_call(payload, timeout_s=config.timeout_s)
        finally:
            logger.info('latency_ms=%s' % int((time.time()-t0)*1000))

    resp = _call_once()
    CACHE[cache_key] = resp
    return resp

llm_call('hello', config=cfg)


## Exercise: persist raw failures (TODO)

Implement the TODO function below.

Goal:

- If the provider call fails (e.g. timeout), persist a short JSON record under `output/`.
- Return the written path.

Checkpoint:

- Trigger a failure (`force_error=True`) and confirm `output/raw_failure.json` exists.

In [ ]:
from pathlib import Path


OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)


def persist_raw_failure_todo(payload: dict, err: Exception, *, filename: str = "raw_failure.json") -> Path:
    # TODO: implement
    out_path = OUTPUT_DIR / filename
    out_path.write_text("TODO\n", encoding="utf-8")
    return out_path


try:
    llm_call("this will fail", config=cfg, force_error=True)
except Exception as e:
    p = persist_raw_failure_todo({"model": cfg.model, "prompt": "this will fail"}, e)
    print("saved failure to", p)

## Appendix: Solutions (peek only after trying)

Reference implementation for `persist_raw_failure_todo`.

In [ ]:
def persist_raw_failure_todo(payload: dict, err: Exception, *, filename: str = "raw_failure.json") -> Path:
    out_path = OUTPUT_DIR / filename
    record = {
        "payload": payload,
        "error_type": type(err).__name__,
        "error": str(err),
    }
    out_path.write_text(json.dumps(record, indent=2), encoding="utf-8")
    return out_path


try:
    llm_call("this will fail", config=cfg, force_error=True)
except Exception as e:
    p = persist_raw_failure_todo({"model": cfg.model, "prompt": "this will fail"}, e, filename="raw_failure_solution.json")
    print("saved failure to", p)